In [49]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Jupyter-friendly version of build_atc_features.py
Safe incremental execution, same logic as original.
"""

from __future__ import annotations
import os, json, re, time, hashlib, random, logging
from typing import List, Dict, Any, Tuple, Optional

import numpy as np
from tqdm.notebook import tqdm
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from sklearn.decomposition import PCA
import joblib
import random as pyrandom
import pickle

logging.basicConfig(level=logging.WARNING)


In [50]:
# I/O
BASE_JSONL_PATH = os.environ.get("BASE_JSONL_PATH", "./data/codenet(python)_gemini_hybrid_line_features.jsonl")

OUTPUT_JSONL_PATH = os.environ.get("OUTPUT_JSONL_PATH", "./limo_atf/segment_python_at_features_sel_prompt.jsonl")
OUTPUT_RAW_EMBED_PATH = os.environ.get("OUTPUT_RAW_EMBED_PATH", "./limo_atf/segment_python_atfeature_sel_prompt_raw_embeddings.jsonl") # save raw embeddings before PCA

PCA_PICKLE_PATH = os.environ.get("PCA_PICKLE_PATH", "./pca/segment_python_atfeature_sel_prompt_pca.pkl")
PCA_META_PATH = os.environ.get("PCA_META_PATH", "./pca/segment_python_atfeature_sel_prompt_pca_meta.json")

TRAIN_DOC_KEYS_FILE = os.environ.get("TRAIN_DOC_KEYS_FILE", "") # optional: a txt file with doc_keys used to fit PCA

# Models
LLM_MODEL_NAME = os.environ.get("LLM_MODEL_NAME", "codellama/CodeLlama-7b-Instruct-hf")
EMBED_MODEL_NAME = os.environ.get("EMBED_MODEL_NAME", "microsoft/codebert-base")

# Generation / Embedding
N_TASKS_PER_SEGMENT = int(os.environ.get("N_TASKS_PER_SEGMENT", "3")) # usually 1 is enough
LLM_MAX_NEW_TOKENS = int(os.environ.get("LLM_MAX_NEW_TOKENS", "128"))
LLM_TEMPERATURE = float(os.environ.get("LLM_TEMPERATURE", "0.8"))
TOP_P = float(os.environ.get("TOP_P", "0.9"))
EMBED_MAX_LEN = int(os.environ.get("EMBED_MAX_LEN", "256"))

# PCA
PCA_DIM = int(os.environ.get("PCA_DIM", "128"))

# Segmentation
SEGMENT_WINDOW_K = int(os.environ.get("SEGMENT_WINDOW_K", "0")) # context segments on each side for prompting
MIN_SEG_LINES = int(os.environ.get("MIN_SEG_LINES", "1")) # merge tiny segments if needed

# Randomness / logging
SEED = int(os.environ.get("SEED", "1234"))
np.random.seed(SEED)

LANGUAGE_HINT = "python"

In [51]:
def sha1_text(s: str) -> str:
    return hashlib.sha1(s.encode("utf-8", errors="ignore")).hexdigest()


def smart_doc_key(rec: Dict[str, Any]) -> str:
    pid = str(rec.get("problem_id", ""))
    did = rec.get("doc_id")
    sha = rec.get("text_sha1")
    if not sha:
        txt = rec.get("text") or ""
        if txt:
            sha = sha1_text(txt)
    if did:
        return f"{pid}::{did}"
    if sha:
        return f"{pid}::{sha}"
    return f"{pid}::row{rec.get('_row_id','')}"


def iter_jsonl(path: str):
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except Exception:
                continue
            obj.setdefault("_row_id", i)
            yield obj


def write_jsonl(path: str, records: List[Dict[str, Any]]):
    with open(path, "w", encoding="utf-8") as f:
        for r in records:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")


def append_jsonl(path: str, records: List[Dict[str, Any]]):
    with open(path, "a", encoding="utf-8") as f:
        for r in records:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")


In [52]:
class TaskGenerator:
    """LLM that turns a code **segment** into a one-sentence approximate task.

    CHANGED: prompt is now segment-centric; removed target-line wording.
    """
    def __init__(self):
        # slight, notebook-friendly defaults; adjust if you have larger GPUs
        max_memory = {0: "20GiB"}
        self.tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL_NAME, use_fast=True)
        self.model = AutoModelForCausalLM.from_pretrained(
            LLM_MODEL_NAME,
            device_map="auto",
            max_memory=max_memory,
            torch_dtype=torch.float16,
        )
        # Avoid the spammy warning by ensuring pad_token_id is set
        if self.tokenizer.pad_token_id is None and self.tokenizer.eos_token_id is not None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id
        self.gen = torch.Generator(device=str(self.model.device)).manual_seed(SEED)

    @staticmethod
    def prompt_segment(segment_text: str, language_hint: Optional[str] = None, context_text: str = "") -> str:
        lang = language_hint or ""
        prog_ctx = f"\n<program_context>\n{context_text}\n</program_context>\n" if context_text else "\n"
        
        return (
            "You are a careful code reviewer analyzing a full program.\n"
            "Your goal is to **infer the design intent** behind each code segment, not to restate its actions.\n"
            "Read the overall program to understand its purpose, then hypothesize **why the TARGET SEGMENT exists**, what issue it solves, or what design goal it fulfills.\n\n"
            "Output: ONE concise English sentence (<=60 words) describing the segment\'s intent or motivation within the program.\n"
            "Prefer action verbs (e.g., validate, parse, dispatch, fetch, cache, format). If context is insufficient, make a\n"
            "best-effort guess based on available information.\n"
            "Avoid implementation detail or pseudocode; focus on purpose and reasoning.\n\n"
            f"<language>{lang}</language>\n"
            f"{prog_ctx}"
            f"<target_segment>\n{segment_text}\n</target_segment>\n"
        )
        
        '''
        return (
            "You are reviewing a full program and one of its code segments.\n"
            "Your goal is to describe both the segment's intended purpose and any design issues or oddities you notice.\n"
            "Consider what the segment is trying to accomplish, the intent of writing this segment, how it interacts with the rest of the program, "
            "and whether its design looks natural or unusual.\n\n"

            "Answer in EXACTLY three short clauses separated by ' || '.\n" \
            "Output ONE SINGLE LINE with NO line breaks.\n"
            "Each clause must be at most 40 words.\n"
            "Use EXACTLY this format:\n" \
            "Intent: <main purpose of the segment> || Interaction: <how it uses inputs/outputs or other code> || Issues or oddities: <missing checks, dead code, inconsistent style, or write 'none obvious'>\n"
            #"Intent: <main purpose of the segment> || "
            #"Interaction: <how it uses inputs/outputs or other code> || "
            #"Issues or oddities: <missing checks, dead code, inconsistent style, or write 'none obvious'>\n\n
            "Do not add any extra text before or after this line. Do not start a new line anywhere."

            "Focus on purpose and design. You may briefly mention key actions or missing checks when they are important "
            "for understanding the design or its problems.\n\n"
            "In Issues or oddities, you MUST mention at least one potential issue, weakness, or strange point, even if it is minor (e.g., vague names, missing checks, unusual structure, unused variables). "
            "If the segment is extremely short (1-2 lines) and truly normal, write \'none obvious\' and briefly say why."


            f"<language>{lang}</language>\n"
            f"{prog_ctx}"
            f"<target_segment>\n{segment_text}\n</target_segment>\n"
        )
        '''

    def wrap(self, user_text: str) -> str:
        if hasattr(self.tokenizer, "apply_chat_template") and callable(self.tokenizer.apply_chat_template):
            msgs = [{"role": "user", "content": user_text}]
            return self.tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
        return f"[INST] {user_text.strip()} [/INST]"

    @torch.inference_mode()
    def gen_tasks(self, segment_text: str, language_hint: Optional[str] = None, context_text: str = "") -> List[str]:
        prompt = self.prompt_segment(segment_text, language_hint, context_text)
        full = self.wrap(prompt)
        in_ids = self.tokenizer(full, return_tensors="pt").to(self.model.device)
        out = self.model.generate(
            **in_ids,
            do_sample=True,
            temperature=LLM_TEMPERATURE,
            max_new_tokens=LLM_MAX_NEW_TOKENS,
            num_return_sequences=N_TASKS_PER_SEGMENT,
            pad_token_id=self.tokenizer.pad_token_id,
            top_p=TOP_P,
        )
        texts = self.tokenizer.batch_decode(out, skip_special_tokens=True)
        # Keep only the assistant parts after the wrap (simple split)
        results = []
        for s in texts:
            if "### Response:" in s:
                s = s.split("### Response:", 1)[-1]
            elif "[/INST]" in s:
                s = s.split("[/INST]", 1)[-1]

            s = s.strip()
            # Sanitise into one line
            s = re.sub(r"\s+", " ", s)
            
            if s:
                #print(f"========== Generated task ==========\n {s}\n====================\n")
                results.append(s)
        # Deduplicate while preserving order
        seen = set()
        uniq = []
        for t in results:
            if t not in seen:
                seen.add(t)
                uniq.append(t)
        return uniq[: N_TASKS_PER_SEGMENT]
    

    @staticmethod
    def prompt_document(program_text: str, language_hint: Optional[str] = None) -> str:
        lang = language_hint or ""
        return (
            "You are a careful code reviewer.\n"
            f"Summarize the overall **design intent** of the entire {lang} program.\n"
            "Read the overall program to understand its purpose, then hypothesize **why the this program exists**, what issue it solves, or what design goal it fulfills\n\n"
            "Output: ONE concise English sentence (<=30 words) describing the intent or motivation of the program.\n"
            "Prefer action verbs (e.g., validate, parse, dispatch, fetch, cache, format). If context is insufficient, make a best-effort guess based on available information.\n"
            "Avoid implementation detail or pseudocode; focus on purpose and reasoning.\n\n"
            "\n\n<program>\n" + program_text + "\n</program>\n"
        )


    @torch.inference_mode()
    def gen_doc_task(self, program_text: str, language_hint: Optional[str] = None) -> List[str]:
        prompt = self.prompt_document(program_text, language_hint)
        full = self.wrap(prompt)
        in_ids = self.tokenizer(full, return_tensors="pt").to(self.model.device)
        out = self.model.generate(
            **in_ids,
            do_sample=True,
            temperature=LLM_TEMPERATURE,
            max_new_tokens=LLM_MAX_NEW_TOKENS,
            num_return_sequences=max(1, N_TASKS_PER_SEGMENT),
            top_p=0.95,
            pad_token_id=self.tokenizer.pad_token_id,
        )
        texts = self.tokenizer.batch_decode(out, skip_special_tokens=True)
        # 复用现有的结果清洗逻辑（和 gen_tasks 一样的那套规则）
        results = []
        for s in texts:
            if "### Response:" in s:
                s = s.split("### Response:", 1)[-1]
            elif "[/INST]" in s:
                s = s.split("[/INST]", 1)[-1]

            s = s.strip()
            s = re.sub(r"\s+", " ", s)
            if s:
                #print(f"========== Generated task ==========\n {s}\n====================\n")
                results.append(s)
        # 去重取前 N 条（通常 1 条就够）
        seen, uniq = set(), []
        for t in results:
            if t not in seen:
                seen.add(t)
                uniq.append(t)
        return uniq[: max(1, N_TASKS_PER_SEGMENT)]


# %%
# --- Task Embedder (CodeBERT) ----------------------------------------------
class TaskEmbedder:
    def __init__(self):
        max_memory = {0: "12GiB"}
        self.tokenizer = AutoTokenizer.from_pretrained(EMBED_MODEL_NAME, use_fast=True)
        self.model = AutoModel.from_pretrained(
            EMBED_MODEL_NAME,
            device_map="auto",
            max_memory=max_memory,
            torch_dtype=torch.float16,
        )
        if self.tokenizer.pad_token_id is None and self.tokenizer.eos_token_id is not None:
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    @torch.inference_mode()
    def encode(self, texts: List[str]) -> np.ndarray:
        enc = self.tokenizer(
            texts, padding=True, truncation=True, max_length=EMBED_MAX_LEN, return_tensors="pt"
        ).to(self.model.device)
        out = self.model(**enc)
        last = out.last_hidden_state
        mask = enc["attention_mask"].unsqueeze(-1)
        summed = (last * mask).sum(dim=1)
        denom = mask.sum(dim=1).clamp(min=1)
        mean_pool = summed / denom
        return mean_pool.float().cpu().numpy()

In [53]:
# Segmenter

BLOCK_START_RE = re.compile(r"^(\s*)(def |class |if |for |while |try:|with |switch\b|function\b|public |private |protected |@) | .*\{\s*$", re.X)


def _is_blank(s: str) -> bool:
    return len(s.strip()) == 0


def _brace_delta(s: str) -> int:
    # naive balance for C-like braces; good enough for heuristic segmentation
    opens = s.count("{")
    closes = s.count("}")
    return opens - closes


def segment_code(lines: List[str], lang_hint: Optional[str] = None, min_seg_lines: int = 1) -> List[Tuple[int, int]]:
    """Return a list of (start_idx, end_idx) for code segments.

    Heuristic: split by blank lines, but keep blocks together until brace depth returns to 0.
    This is language-agnostic and robust enough for mixed repos.
    """
    n = len(lines)
    segs: List[Tuple[int, int]] = []
    i = 0
    while i < n:
        # skip leading blanks
        while i < n and _is_blank(lines[i]):
            i += 1
        if i >= n:
            break
        start = i
        depth = 0
        j = i
        while j < n:
            s = lines[j]
            depth += _brace_delta(s)
            # if blank and depth==0 -> boundary
            if _is_blank(s) and depth == 0 and j > start:
                # trim trailing blanks
                k = j - 1
                while k >= start and _is_blank(lines[k]):
                    k -= 1
                if k >= start:
                    segs.append((start, k))
                j += 1
                break
            j += 1
        else:
            # reached EOF
            k = n - 1
            while k >= start and _is_blank(lines[k]):
                k -= 1
            if k >= start:
                segs.append((start, k))
            break
        i = j

    # merge tiny segments forward if requested
    if min_seg_lines > 1 and segs:
        merged: List[Tuple[int, int]] = []
        buf_start, buf_end = segs[0]
        for (s, e) in segs[1:]:
            if (buf_end - buf_start + 1) < min_seg_lines:
                # merge with next
                buf_end = e
            else:
                merged.append((buf_start, buf_end))
                buf_start, buf_end = s, e
        merged.append((buf_start, buf_end))
        segs = merged

    return segs


In [54]:
records = list(iter_jsonl(BASE_JSONL_PATH))
print(f"Loaded {len(records)} records from {BASE_JSONL_PATH}")

Loaded 2787 records from ./data/codenet(python)_gemini_hybrid_line_features.jsonl


# ***Main Logic***

### ***Extracting Segment-Level (Code block) Approximated Task Features***

In [55]:
taskgen = TaskGenerator()
embedder = TaskEmbedder()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [56]:
all_segments_meta: List[Dict[str, Any]] = [] # per segment
all_segment_vecs: List[np.ndarray] = [] # raw vectors (pre-PCA), one per segment

In [57]:
# CUDA check
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

print("LLM devices:", {p.device.type for p in taskgen.model.parameters()})
print("Embedder devices:", {p.device.type for p in embedder.model.parameters()})


CUDA available: True
GPU: Quadro RTX 8000
LLM devices: {'cuda'}
Embedder devices: {'cuda'}


In [58]:
pbar_docs = tqdm(
    records, 
    desc="Docs", 
    dynamic_ncols=True, 
    position=0
)

open(OUTPUT_RAW_EMBED_PATH, "w", encoding="utf-8").close()  # clear file

for doc_idx, rec in enumerate(pbar_docs):
    doc_key = smart_doc_key(rec)
    pid = str(rec.get("problem_id", ""))
    lines = rec.get("lines") or (rec.get("text") or "").splitlines()
    lang_hint = LANGUAGE_HINT

    segs = segment_code(lines, lang_hint, min_seg_lines=MIN_SEG_LINES)

    full_program_text = rec.get("text") or "\n".join(lines)

    pbar_segs = tqdm(
        total=len(segs),
        desc=f"Doc {doc_idx+1}/{len(records)} segments",
        dynamic_ncols=True,
        position=1,
        leave=False,
        mininterval=0.1,
    )

    # Build simple neighbor context (optional)
    seg_count = 0

    # per-document debug cap
    debug_prompts_printed = 0

    for seg_id, (s, e) in enumerate(segs):
        # Assemble prompt inputs
        seg_text = "\n".join(lines[s : e + 1])
        '''
        ctx_parts = []
        if SEGMENT_WINDOW_K > 0:
            left = max(0, seg_id - SEGMENT_WINDOW_K)
            right = min(len(segs) - 1, seg_id + SEGMENT_WINDOW_K)
            for k in range(left, right + 1):
                if k == seg_id:
                    continue
                ss, ee = segs[k]
                ctx_parts.append("\n".join(lines[ss : ee + 1]))
        ctx_text = "\n\n".join(ctx_parts)
        '''
        ctx_text = full_program_text

        # Prepare a debug copy of the exact prompt (without chat wrap)
        debug_prompt = taskgen.prompt_segment(seg_text, lang_hint, ctx_text)

        #print(taskgen.tokenizer.chat_template)
        tasks = taskgen.gen_tasks(seg_text, lang_hint, ctx_text)
        if not tasks:
            tasks = ["Perform the described code segment task."]
        vecs = embedder.encode(tasks)
        vec = vecs.mean(axis=0)

        '''
        # debugging statements
        if debug_prompts_printed < 5:
            print(f"\n[DEBUG] doc_key={doc_key} seg_id={seg_id} lines={s}-{e}")
            print("----- PROMPT BEGIN -----")
            print(debug_prompt)
            print("----- PROMPT END -------")
            print("----- APPROX TASK(S) -----")
            for t_i, t in enumerate(tasks, 1):
                print(f"{t_i}. {t}")
            print("----- END --------------\n")
            debug_prompts_printed += 1

        if seg_count < 20:
            print(f"seg_text ({s}-{e}):============\n{seg_text}\n===end of segment===\n")
            seg_count += 1
        '''

        '''
        all_segments_meta.append({
            "doc_key": doc_key,
            "problem_id": pid,
            "segment_id": seg_id,
            "seg_start": s,
            "seg_end": e,
            "n_lines": (e - s + 1),
            "n_tasks": len(tasks),
        })
        all_segment_vecs.append(vec)
        '''

        record = {
            "doc_key": doc_key,
            "problem_id": pid,
            "segment_id": seg_id,
            "seg_start": s,
            "seg_end": e,
            "n_lines": (e - s + 1),
            "n_tasks": len(tasks),
            "vec_raw": [float(x) for x in vec.tolist()],
        }
        append_jsonl(OUTPUT_RAW_EMBED_PATH, [record])

        pbar_segs.update(1)

    pbar_segs.close()

print(f"Collected {len(all_segment_vecs)} segment vectors across {len(records)} docs.")
print(f"Saved raw embeddings to {OUTPUT_RAW_EMBED_PATH}")

Docs:   0%|          | 0/2787 [00:00<?, ?it/s]

Doc 1/2787 segments:   0%|          | 0/6 [00:00<?, ?it/s]

Doc 2/2787 segments:   0%|          | 0/7 [00:00<?, ?it/s]

Doc 3/2787 segments:   0%|          | 0/7 [00:00<?, ?it/s]

Doc 4/2787 segments:   0%|          | 0/5 [00:00<?, ?it/s]

...

Doc 2784/2787 segments:   0%|          | 0/5 [00:00<?, ?it/s]

Doc 2785/2787 segments:   0%|          | 0/4 [00:00<?, ?it/s]

Doc 2786/2787 segments:   0%|          | 0/7 [00:00<?, ?it/s]

Doc 2787/2787 segments:   0%|          | 0/3 [00:00<?, ?it/s]

Collected 0 segment vectors across 2787 docs.
Saved raw embeddings to ./limo_atf/segment_python_atfeature_sel_prompt_raw_embeddings.jsonl


### ***Transferring BERT's RAW 768-dims Embeddings into 128-dims for Classifier***

In [59]:
import os, json, math, gc
import numpy as np
from tqdm.notebook import tqdm
from sklearn.decomposition import IncrementalPCA
import joblib

# ====== 路径参数（按需改）======
RAW_EMB_PATH = os.environ.get("OUTPUT_RAW_EMBED_PATH", "./limo_atf/segment_python_atfeature_sel_prompt_raw_embeddings.jsonl") # 你给的示例文件，实际跑全量时改回你的 raw embeddings 路径
OUT_SEG_PCA_JSONL = os.environ.get("OUT_SEG_PCA_JSONL", "./limo_atf/segment_python_atfeature_sel_prompt_pca128.jsonl")
PCA_PICKLE_PATH = os.environ.get("PCA_PICKLE_PATH", "./pca/segment_python_atfeature_sel_prompt_pca128.pkl")

# ====== PCA 参数（可改）======
N_COMPONENTS = 128
BATCH_SIZE   = 4096   # 按显存/内存酌情调大/调小
DTYPE        = np.float32
ROUND_DEC    = 12      # 写 JSON 时保留的小数位，空间 vs 精度自己权衡

# 如果只想用“训练集文档”去拟合 PCA（更科学），可以把 doc_key 放在一个 txt 里
TRAIN_DOC_KEYS_FILE = os.environ.get("TRAIN_DOC_KEYS_FILE", "")  # 可留空
if TRAIN_DOC_KEYS_FILE and os.path.exists(TRAIN_DOC_KEYS_FILE):
    with open(TRAIN_DOC_KEYS_FILE, "r", encoding="utf-8") as f:
        TRAIN_DOC_KEYS = set(x.strip() for x in f if x.strip())
else:
    TRAIN_DOC_KEYS = None

def iter_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line: 
                continue
            yield json.loads(line)

def record_to_vec(rec):
    # Raw 里是 rec["vec_raw"]，长度通常 768（CodeBERT-base）
    return np.asarray(rec["vec_raw"], dtype=DTYPE)

def want_for_fit(rec):
    # 若提供了训练集 doc_key 白名单，则只用这些样本进行 partial_fit
    return (TRAIN_DOC_KEYS is None) or (rec.get("doc_key") in TRAIN_DOC_KEYS)


In [60]:
ipca = IncrementalPCA(n_components=N_COMPONENTS)

buf = []
cnt_fit = 0

for rec in tqdm(iter_jsonl(RAW_EMB_PATH), desc="Fitting PCA (pass-1)"):
    if not want_for_fit(rec):
        continue
    v = record_to_vec(rec)
    buf.append(v)
    if len(buf) == BATCH_SIZE:
        X = np.vstack(buf)
        ipca.partial_fit(X)
        cnt_fit += len(buf)
        buf.clear()

if buf:
    X = np.vstack(buf)
    ipca.partial_fit(X)
    cnt_fit += len(buf)
    buf.clear()

print(f"[PCA] partial_fit samples: {cnt_fit}")
joblib.dump(ipca, PCA_PICKLE_PATH)
print(f"[PCA] saved to: {PCA_PICKLE_PATH}")

# 解释方差占比（大概看下 128 维保留了多少信息）
explained = getattr(ipca, "explained_variance_ratio_", None)
if explained is not None:
    print(f"[PCA] explained variance (sum of 128): {explained.sum():.4f}")


Fitting PCA (pass-1): 0it [00:00, ?it/s]

[PCA] partial_fit samples: 13794
[PCA] saved to: ./pca/segment_python_atfeature_sel_prompt_pca128.pkl
[PCA] explained variance (sum of 128): 0.9404


In [61]:
# 先清空输出文件
os.makedirs(os.path.dirname(OUT_SEG_PCA_JSONL), exist_ok=True)
open(OUT_SEG_PCA_JSONL, "w", encoding="utf-8").close()

In [62]:
buf = []
meta_buf = []
written = 0

def flush_transform_write():
    global written
    if not buf: 
        return
    X = np.vstack(buf)
    Z = ipca.transform(X).astype(DTYPE)       # [batch, 128]
    with open(OUT_SEG_PCA_JSONL, "a", encoding="utf-8") as wf:
        for meta, z in zip(meta_buf, Z):
            out = {
                # —— 保留你后面可能会用到的定位信息 —— 
                "doc_key":   meta["doc_key"],
                "problem_id":meta["problem_id"],
                "segment_id":meta["segment_id"],
                "seg_start": meta["seg_start"],
                "seg_end":   meta["seg_end"],
                "n_lines":   meta["n_lines"],
                # —— 新的 128D 向量 —— 
                "vec_atc128":[round(float(x), ROUND_DEC) for x in z.tolist()],
            }
            wf.write(json.dumps(out, ensure_ascii=False) + "\n")
            written += 1
    buf.clear()
    meta_buf.clear()
    gc.collect()

for rec in tqdm(iter_jsonl(RAW_EMB_PATH), desc="Transform & Write (pass-2)"):
    v = record_to_vec(rec)
    buf.append(v)
    # 只保留必要的元信息，原始 vec_raw 不再写回
    meta_buf.append({
        "doc_key":   rec.get("doc_key"),
        "problem_id":rec.get("problem_id"),
        "segment_id":rec.get("segment_id"),
        "seg_start": rec.get("seg_start"),
        "seg_end":   rec.get("seg_end"),
        "n_lines":   rec.get("n_lines"),
    })
    if len(buf) == BATCH_SIZE:
        flush_transform_write()

flush_transform_write()
print(f"[WRITE] segments written: {written}")
print(f"[OUT] {OUT_SEG_PCA_JSONL}")


Transform & Write (pass-2): 0it [00:00, ?it/s]

[WRITE] segments written: 13794
[OUT] ./limo_atf/segment_python_atfeature_sel_prompt_pca128.jsonl


In [63]:
import json, numpy as np
from tqdm.notebook import tqdm

IN_PCA_JSONL = "./limo_atf/segment_python_atfeature_pca128.jsonl"

n, bad = 0, 0
for line in tqdm(open(IN_PCA_JSONL, "r", encoding="utf-8"), desc="Check 128-d"):
    rec = json.loads(line)
    v = rec.get("vec_atc128", [])
    if not isinstance(v, list) or len(v) != 128:
        bad += 1
    n += 1

print(f"total records: {n}, bad: {bad}")
assert bad == 0, "发现不是128维的记录，请先清洗掉这些行"


Check 128-d: 0it [00:00, ?it/s]

total records: 13794, bad: 0


In [64]:
# segment→line（sidecar）生成器：按照 ccfeatures 策略
import os, json, hashlib
import numpy as np
from collections import defaultdict
from tqdm.notebook import tqdm
from datetime import datetime

# ===== 输入：你已生成的 128 维 segment 级 JSONL =====
IN_PCA_JSONL = "./limo_atf/segment_python_atfeature_sel_prompt_pca128.jsonl"  # 改成你的路径也行

# ===== 输出：与 ccfeatures 风格一致的 sidecar 目录结构 =====
OUT_DIR = "./limo_atf/great_data_sel_prompt"         # 每个文档会有一个 .npy
INDEX_JSON = os.path.join(OUT_DIR, "index.json")

# ===== 行为参数 =====
EMB_DIM = 128                 # 你已确定 128
DTYPE_SAVE = "float32"        # 可选 "float32" / "float16"
OVERLAP_MODE = "average"      # "average"（推荐，与 ccfeatures 一致无歧义）或 "last"

os.makedirs(OUT_DIR, exist_ok=True)

def _safe_key(doc_key: str) -> str:
    """把 doc_key 转成稳定可用的文件名（避免奇怪字符/过长）。"""
    return hashlib.sha1(doc_key.encode("utf-8")).hexdigest()[:16]

def _as_dtype(arr: np.ndarray, dtype_name: str) -> np.ndarray:
    return arr.astype(np.float16 if dtype_name == "float16" else np.float32, copy=False)


In [65]:
# 第 1 遍：把同一 doc_key 的 segment 收集起来，并统计每个 doc 的最大行号（决定矩阵高度）
groups = defaultdict(list)   # doc_key -> [(s, e, vec128), ...]
meta   = {}                  # doc_key -> {"problem_id": ..., "max_end": ...}

with open(IN_PCA_JSONL, "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="Group segments by doc_key"):
        rec = json.loads(line)
        dk  = rec["doc_key"]
        s, e = int(rec["seg_start"]), int(rec["seg_end"])
        vec = np.asarray(rec["vec_atc128"], dtype=np.float32)  # 先用 fp32 聚合更稳
        if vec.shape[0] != EMB_DIM:
            raise ValueError(f"Expect {EMB_DIM}-D, got {vec.shape} for doc_key={dk}")
        groups[dk].append((s, e, vec))
        info = meta.setdefault(dk, {
            "problem_id": rec.get("problem_id", ""),
            "max_end": -1
        })
        if e > info["max_end"]:
            info["max_end"] = e

print(f"docs collected: {len(groups)}")


Group segments by doc_key: 0it [00:00, ?it/s]

docs collected: 2752


In [66]:
# 第 2 遍：对每个 doc 做广播
index = {}
for dk, segs in tqdm(groups.items(), desc="Write per-doc line npy"):
    L = int(meta[dk]["max_end"]) + 1
    if L <= 0:
        # 空文档，落一个空矩阵也行；这里直接跳过（看你数据是否存在这种情况）
        continue

    # A: 行级累加矩阵；C: 计数矩阵（用于平均）
    A = np.zeros((L, EMB_DIM), dtype=np.float32)
    C = np.zeros((L, 1),       dtype=np.int32)

    if OVERLAP_MODE == "average":
        # 对每个 segment 的范围，做向量累加 & 计数 +1
        for s, e, vec in segs:
            s = max(0, s)
            e = min(L - 1, e)
            if e < s: 
                continue
            A[s:e+1, :] += vec
            C[s:e+1, :] += 1

        # 把累加结果除以计数，得到平均；无覆盖的行为 0（C=0）
        mask = (C > 0)
        A[mask[:, 0]] /= C[mask][:, None]

    elif OVERLAP_MODE == "last":
        # 直接覆盖：后面的 segment 覆盖前面的（不建议，除非你确认没有重叠）
        for s, e, vec in segs:
            s = max(0, s)
            e = min(L - 1, e)
            if e < s:
                continue
            A[s:e+1, :] = vec
    else:
        raise ValueError("OVERLAP_MODE must be 'average' or 'last'")

    # 保存为 .npy（体积可选 float16/float32）
    key = _safe_key(dk)
    npy_path = os.path.join(OUT_DIR, f"{key}.npy")
    np.save(npy_path, _as_dtype(A, DTYPE_SAVE))

    # 写入索引
    index[dk] = {
        "problem_id": meta[dk]["problem_id"],
        "n_lines": int(L),
        "dim": EMB_DIM,
        "dtype": DTYPE_SAVE,
        "path": npy_path
    }

# 总索引
with open(INDEX_JSON, "w", encoding="utf-8") as wf:
    json.dump({
        "created_at": datetime.utcnow().isoformat()+"Z",
        "overlap_mode": OVERLAP_MODE,
        "docs": index
    }, wf, ensure_ascii=False, indent=2)

print(f"[DONE] docs: {len(index)}")
print(f"[INDEX] {INDEX_JSON}")


Write per-doc line npy:   0%|          | 0/2752 [00:00<?, ?it/s]

[DONE] docs: 2752
[INDEX] ./limo_atf/great_data_sel_prompt/index.json


In [67]:
# 读取 sidecar 中某个 doc_key 的行级 AT 矩阵
def load_atc_sidecar(index_json: str, doc_key: str):
    with open(index_json, "r", encoding="utf-8") as f:
        idx = json.load(f)
    info = idx["docs"][doc_key]
    A = np.load(info["path"])   # shape: [n_lines, 128]，dtype 可能是 fp16/fp32
    return A, info

# 和 ccfeature 在最后一维拼接（按最小长度对齐）
def concat_cc_and_atc(cc_mat: np.ndarray, at_mat: np.ndarray):
    """
    cc_mat: [L1, CcDim]
    at_mat: [L2, 128]
    返回: [min(L1,L2), CcDim+128]
    """
    L = min(cc_mat.shape[0], at_mat.shape[0])
    if at_mat.dtype != np.float32:
        at_mat = at_mat.astype(np.float32, copy=False)
    return np.concatenate([cc_mat[:L], at_mat[:L]], axis=1)


In [68]:
# 随机抽一个 doc 做检查
any_doc = next(iter(index))
A = np.load(index[any_doc]["path"])
print(any_doc, A.shape, A.dtype, A.min(), A.max())
assert A.shape[1] == 128


p03166::d5d6e3d0311f85dd9eeb15a32b9f1c8a7f90b330 (30, 128) float32 -1.4091984 0.8849057


### ***Below is for generating 256dim `at_features`, not 128***

In [ ]:
import os, json, math, gc
import numpy as np
from tqdm.notebook import tqdm
from sklearn.decomposition import IncrementalPCA
import joblib

# ====== 路径参数（按需改）======
RAW_EMB_PATH = os.environ.get("OUTPUT_RAW_EMBED_PATH", "./limo_atf/segment_python_atfeature_raw_embeddings.jsonl") # raw embeddings 路径
OUT_SEG_PCA_JSONL = os.environ.get("OUT_SEG_PCA_JSONL", "./limo_atf/segment_python_atfeature_pca256.jsonl")
PCA_PICKLE_PATH = os.environ.get("PCA_PICKLE_PATH", "./pca/segment_python_atfeature_pca256.pkl")

# ====== PCA 参数（可改）======
N_COMPONENTS = 256
BATCH_SIZE   = 4096   # 按显存/内存酌情调大/调小
DTYPE        = np.float32
ROUND_DEC    = 12      # 写 JSON 时保留的小数位，空间 vs 精度自己权衡

# 如果只想用“训练集文档”去拟合 PCA（更科学），可以把 doc_key 放在一个 txt 里
TRAIN_DOC_KEYS_FILE = os.environ.get("TRAIN_DOC_KEYS_FILE", "")  # 可留空
if TRAIN_DOC_KEYS_FILE and os.path.exists(TRAIN_DOC_KEYS_FILE):
    with open(TRAIN_DOC_KEYS_FILE, "r", encoding="utf-8") as f:
        TRAIN_DOC_KEYS = set(x.strip() for x in f if x.strip())
else:
    TRAIN_DOC_KEYS = None

def iter_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line: 
                continue
            yield json.loads(line)

def record_to_vec(rec):
    # Raw 里是 rec["vec_raw"]，长度通常 768（CodeBERT-base）
    return np.asarray(rec["vec_raw"], dtype=DTYPE)

def want_for_fit(rec):
    # 若提供了训练集 doc_key 白名单，则只用这些样本进行 partial_fit
    return (TRAIN_DOC_KEYS is None) or (rec.get("doc_key") in TRAIN_DOC_KEYS)


In [ ]:
ipca = IncrementalPCA(n_components=N_COMPONENTS)

buf = []
cnt_fit = 0

for rec in tqdm(iter_jsonl(RAW_EMB_PATH), desc="Fitting PCA (pass-1)"):
    if not want_for_fit(rec):
        continue
    v = record_to_vec(rec)
    buf.append(v)
    if len(buf) == BATCH_SIZE:
        X = np.vstack(buf)
        ipca.partial_fit(X)
        cnt_fit += len(buf)
        buf.clear()

if buf:
    X = np.vstack(buf)
    ipca.partial_fit(X)
    cnt_fit += len(buf)
    buf.clear()

print(f"[PCA] partial_fit samples: {cnt_fit}")
joblib.dump(ipca, PCA_PICKLE_PATH)
print(f"[PCA] saved to: {PCA_PICKLE_PATH}")

# 解释方差占比（大概看下 256 维保留了多少信息）
explained = getattr(ipca, "explained_variance_ratio_", None)
if explained is not None:
    print(f"[PCA] explained variance (sum of 256): {explained.sum():.4f}")


In [ ]:
# 先清空输出文件
os.makedirs(os.path.dirname(OUT_SEG_PCA_JSONL), exist_ok=True)
open(OUT_SEG_PCA_JSONL, "w", encoding="utf-8").close()

buf = []
meta_buf = []
written = 0

def flush_transform_write():
    global written
    if not buf: 
        return
    X = np.vstack(buf)
    Z = ipca.transform(X).astype(DTYPE)       # [batch, 256]
    with open(OUT_SEG_PCA_JSONL, "a", encoding="utf-8") as wf:
        for meta, z in zip(meta_buf, Z):
            out = {
                # —— 保留你后面可能会用到的定位信息 —— 
                "doc_key":   meta["doc_key"],
                "problem_id":meta["problem_id"],
                "segment_id":meta["segment_id"],
                "seg_start": meta["seg_start"],
                "seg_end":   meta["seg_end"],
                "n_lines":   meta["n_lines"],
                # —— 新的 256D 向量 —— 
                "vec_atc256":[round(float(x), ROUND_DEC) for x in z.tolist()],
            }
            wf.write(json.dumps(out, ensure_ascii=False) + "\n")
            written += 1
    buf.clear()
    meta_buf.clear()
    gc.collect()

for rec in tqdm(iter_jsonl(RAW_EMB_PATH), desc="Transform & Write (pass-2)"):
    v = record_to_vec(rec)
    buf.append(v)
    # 只保留必要的元信息，原始 vec_raw 不再写回
    meta_buf.append({
        "doc_key":   rec.get("doc_key"),
        "problem_id":rec.get("problem_id"),
        "segment_id":rec.get("segment_id"),
        "seg_start": rec.get("seg_start"),
        "seg_end":   rec.get("seg_end"),
        "n_lines":   rec.get("n_lines"),
    })
    if len(buf) == BATCH_SIZE:
        flush_transform_write()

flush_transform_write()
print(f"[WRITE] segments written: {written}")
print(f"[OUT] {OUT_SEG_PCA_JSONL}")


In [ ]:
import json, numpy as np
from tqdm.notebook import tqdm

IN_PCA_JSONL = "./limo_atf/segment_python_atfeature_pca256.jsonl"

n, bad = 0, 0
for line in tqdm(open(IN_PCA_JSONL, "r", encoding="utf-8"), desc="Check 256-d"):
    rec = json.loads(line)
    v = rec.get("vec_atc256", [])
    if not isinstance(v, list) or len(v) != 256:
        bad += 1
    n += 1

print(f"total records: {n}, bad: {bad}")
assert bad == 0, "发现不是256维的记录, 先清洗掉这些行"


In [ ]:
# segment→line（sidecar）生成器：按照 ccfeatures 策略
import os, json, hashlib
import numpy as np
from collections import defaultdict
from tqdm.notebook import tqdm
from datetime import datetime

# ===== 输入：已生成的 256 维 segment 级 JSONL =====
IN_PCA_JSONL = "./limo_atf/segment_python_atfeature_pca256.jsonl"  # 改成你的路径也行

# ===== 输出：与 ccfeatures 风格一致的 sidecar 目录结构 =====
OUT_DIR = "./limo_atf/great_data_256"         # 每个文档会有一个 .npy
INDEX_JSON = os.path.join(OUT_DIR, "index.json")

# ===== 行为参数 =====
EMB_DIM = 256                 # 你已确定 256
DTYPE_SAVE = "float32"        # 可选 "float32" / "float16"
OVERLAP_MODE = "average"      # "average"（推荐，与 ccfeatures 一致无歧义）或 "last"

os.makedirs(OUT_DIR, exist_ok=True)

def _safe_key(doc_key: str) -> str:
    """把 doc_key 转成稳定可用的文件名（避免奇怪字符/过长）。"""
    return hashlib.sha1(doc_key.encode("utf-8")).hexdigest()[:16]

def _as_dtype(arr: np.ndarray, dtype_name: str) -> np.ndarray:
    return arr.astype(np.float16 if dtype_name == "float16" else np.float32, copy=False)


In [ ]:
# 第 1 遍：把同一 doc_key 的 segment 收集起来，并统计每个 doc 的最大行号（决定矩阵高度）
groups = defaultdict(list)   # doc_key -> [(s, e, vec256), ...]
meta   = {}                  # doc_key -> {"problem_id": ..., "max_end": ...}

with open(IN_PCA_JSONL, "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="Group segments by doc_key"):
        rec = json.loads(line)
        dk  = rec["doc_key"]
        s, e = int(rec["seg_start"]), int(rec["seg_end"])
        vec = np.asarray(rec["vec_atc256"], dtype=np.float32)  # 先用 fp32 聚合更稳
        if vec.shape[0] != EMB_DIM:
            raise ValueError(f"Expect {EMB_DIM}-D, got {vec.shape} for doc_key={dk}")
        groups[dk].append((s, e, vec))
        info = meta.setdefault(dk, {
            "problem_id": rec.get("problem_id", ""),
            "max_end": -1
        })
        if e > info["max_end"]:
            info["max_end"] = e

print(f"docs collected: {len(groups)}")


In [ ]:
# 第 2 遍：对每个 doc 做广播
index = {}
for dk, segs in tqdm(groups.items(), desc="Write per-doc line npy"):
    L = int(meta[dk]["max_end"]) + 1
    if L <= 0:
        # 空文档，落一个空矩阵也行；这里直接跳过（看你数据是否存在这种情况）
        continue

    # A: 行级累加矩阵；C: 计数矩阵（用于平均）
    A = np.zeros((L, EMB_DIM), dtype=np.float32)
    C = np.zeros((L, 1),       dtype=np.int32)

    if OVERLAP_MODE == "average":
        # 对每个 segment 的范围，做向量累加 & 计数 +1
        for s, e, vec in segs:
            s = max(0, s)
            e = min(L - 1, e)
            if e < s: 
                continue
            A[s:e+1, :] += vec
            C[s:e+1, :] += 1

        # 把累加结果除以计数，得到平均；无覆盖的行为 0（C=0）
        mask = (C > 0)
        A[mask[:, 0]] /= C[mask][:, None]

    elif OVERLAP_MODE == "last":
        # 直接覆盖：后面的 segment 覆盖前面的（不建议，除非你确认没有重叠）
        for s, e, vec in segs:
            s = max(0, s)
            e = min(L - 1, e)
            if e < s:
                continue
            A[s:e+1, :] = vec
    else:
        raise ValueError("OVERLAP_MODE must be 'average' or 'last'")

    # 保存为 .npy（体积可选 float16/float32）
    key = _safe_key(dk)
    npy_path = os.path.join(OUT_DIR, f"{key}.npy")
    np.save(npy_path, _as_dtype(A, DTYPE_SAVE))

    # 写入索引
    index[dk] = {
        "problem_id": meta[dk]["problem_id"],
        "n_lines": int(L),
        "dim": EMB_DIM,
        "dtype": DTYPE_SAVE,
        "path": npy_path
    }

# 总索引
with open(INDEX_JSON, "w", encoding="utf-8") as wf:
    json.dump({
        "created_at": datetime.utcnow().isoformat()+"Z",
        "overlap_mode": OVERLAP_MODE,
        "docs": index
    }, wf, ensure_ascii=False, indent=2)

print(f"[DONE] docs: {len(index)}")
print(f"[INDEX] {INDEX_JSON}")


In [ ]:
# 读取 sidecar 中某个 doc_key 的行级 AT 矩阵
def load_atc_sidecar(index_json: str, doc_key: str):
    with open(index_json, "r", encoding="utf-8") as f:
        idx = json.load(f)
    info = idx["docs"][doc_key]
    A = np.load(info["path"])   # shape: [n_lines, 256]，dtype 可能是 fp16/fp32
    return A, info

# 和 ccfeature 在最后一维拼接（按最小长度对齐）
def concat_cc_and_atc(cc_mat: np.ndarray, at_mat: np.ndarray):
    """
    cc_mat: [L1, CcDim]
    at_mat: [L2, 256]
    返回: [min(L1,L2), CcDim+256]
    """
    L = min(cc_mat.shape[0], at_mat.shape[0])
    if at_mat.dtype != np.float32:
        at_mat = at_mat.astype(np.float32, copy=False)
    return np.concatenate([cc_mat[:L], at_mat[:L]], axis=1)


In [ ]:
# 随机抽一个 doc 做检查
any_doc = next(iter(index))
A = np.load(index[any_doc]["path"])
print(any_doc, A.shape, A.dtype, A.min(), A.max())
assert A.shape[1] == 256


### ***Below is for generating doc-level at_feature (128 dimensions)***

In [ ]:
# ===================== DOC-LEVEL ATF PIPELINE =====================
# 配置
OUT_DIR_DOC = "./limo_atf/great_data_doc"     # 新的 sidecar 输出目录
INDEX_JSON_DOC = os.path.join(OUT_DIR_DOC, "index.json")
RAW_EMBED_DOC = "./limo_atf/document_python_atfeature_raw_embeddings.jsonl"
PCA_PICKLE_DOC = "./pca/document_python_atfeature_pca128.pkl"
PCA128_JSONL_DOC = "./limo_atf/document_python_atfeature_pca128.jsonl"

AT_DIM = 128         # 和 DataLoader 的期望保持一致
SAVE_DTYPE = "float32"  # 文件更小，训练再转回 fp32


In [ ]:
# 清空旧文件
open(RAW_EMBED_DOC, "w", encoding="utf-8").close()
open(PCA128_JSONL_DOC, "w", encoding="utf-8").close()
os.makedirs(os.path.dirname(PCA_PICKLE_DOC), exist_ok=True)

In [ ]:


taskgen = TaskGenerator()
embedder = TaskEmbedder()

from sklearn.decomposition import IncrementalPCA
import joblib
import time
from collections import defaultdict

def _safe_key(doc_key: str) -> str:
    return hashlib.sha1(doc_key.encode("utf-8")).hexdigest()[:16]

def _as_dtype(arr: np.ndarray, dtype_name: str) -> np.ndarray:
    return arr.astype(np.float16 if dtype_name == "float16" else np.float32, copy=False)

In [ ]:
# ------- A) 逐文档生成 RAW 向量（CodeBERT, 768-d） -------
raw_vecs = []   # 仅用于统计/局部拟合；真正的向量写到 jsonl
DOCS = records  # 复用你上游已经准备好的 records（和原脚本一致）

#tqdm(groups.items(), desc="Write per-doc line npy")
for rec in tqdm(DOCS, desc="Doc-level ATF (raw)"):
    doc_key = smart_doc_key(rec)
    pid = str(rec.get("problem_id", ""))
    lines = rec.get("lines") or (rec.get("text") or "").splitlines()
    full_text = rec.get("text") or "\n".join(lines)
    if not full_text.strip():
        # 空文档兜底
        vec_raw = np.zeros((768,), dtype=np.float32)
    else:
        # 1) 让 LLM 产“一句话设计意图”
        tasks = taskgen.gen_doc_task(full_text, LANGUAGE_HINT)
        if not tasks:
            tasks = ["A program that solves a problem with concise input/output; high-level intent unspecified."]
        # 2) CodeBERT 做均值池化嵌入 → 768 维
        V = embedder.encode(tasks)     # [k, 768]
        vec_raw = V.mean(axis=0)       # [768]
    raw_vecs.append(vec_raw)
    append_jsonl(RAW_EMBED_DOC, [{
        "doc_key": doc_key,
        "problem_id": pid,
        "n_lines": len(lines),
        "vec_raw": vec_raw.tolist()
    }])


In [ ]:
# ------- B) 拟合 PCA→128 维，并写出 pca128.jsonl -------
#tqdm(DOCS, desc="Doc-level ATF (raw)")
ipca = IncrementalPCA(n_components=AT_DIM, batch_size=512)
cnt_fit = 0
# 第一遍 partial_fit
prog_bar = tqdm(range(0, len(raw_vecs), 512), desc="Fitting PCA (doc-level)")
for chunk_start in range(0, len(raw_vecs), 512):
    chunk = np.stack(raw_vecs[chunk_start: chunk_start+512], axis=0)
    ipca.partial_fit(chunk)
    cnt_fit += len(chunk)
    prog_bar.update(1)
prog_bar.close()

print(f"[PCA/doc] partial_fit samples: {cnt_fit}")
joblib.dump(ipca, PCA_PICKLE_DOC)

# 第二遍 transform + 写出 jsonl
with open(RAW_EMBED_DOC, "r", encoding="utf-8") as rf, \
     open(PCA128_JSONL_DOC, "w", encoding="utf-8") as wf:
    for line in tqdm(rf, desc="PCA->128"):
        rec = json.loads(line)
        v = np.array(rec["vec_raw"], dtype=np.float32)[None, :]  # [1,768]
        vec128 = ipca.transform(v)[0].astype(np.float32)         # [128]
        rec_out = {
            "doc_key": rec["doc_key"],
            "problem_id": rec["problem_id"],
            "n_lines": rec["n_lines"],
            "vec_atc128": vec128.tolist()
        }
        wf.write(json.dumps(rec_out, ensure_ascii=False) + "\n")

In [ ]:
# ------- C) 生成 sidecar：每个文档一份 [L, 128]，每行同向量 -------
index = {}
with open(PCA128_JSONL_DOC, "r", encoding="utf-8") as f:
    for line in tqdm(f, desc="Build DOC sidecar"):
        rec = json.loads(line)
        doc_key = rec["doc_key"]
        pid = rec["problem_id"]
        L = int(rec["n_lines"])
        v = np.array(rec["vec_atc128"], dtype=np.float32)  # [128]

        if v.shape[0] != AT_DIM:
            raise ValueError(f"AT dim mismatch: got {v.shape[0]}, expect {AT_DIM}")

        # 重复到行： [L, 128]
        mat = np.broadcast_to(v[None, :], (L, AT_DIM)).copy()
        mat = _as_dtype(mat, SAVE_DTYPE)

        base = _safe_key(doc_key)
        npy_path = os.path.join(OUT_DIR_DOC, base + ".npy")
        np.save(npy_path, mat)

        index[doc_key] = {
            "problem_id": pid,
            "n_lines": L,
            "dim": AT_DIM,
            "dtype": SAVE_DTYPE,
            "path": npy_path
        }


In [ ]:
# 写 index.json
with open(INDEX_JSON_DOC, "w", encoding="utf-8") as wf:
    json.dump({
        "created_at": time.strftime("%Y-%m-%d %H:%M:%S"),
        "overlap_mode": "document",   # 仅信息标注
        "docs": index
    }, wf, ensure_ascii=False, indent=2)

print(f"[DONE] doc-level sidecar: {len(index)} docs")
print(f"[INDEX] {INDEX_JSON_DOC}")
# =================== END DOC-LEVEL ATF PIPELINE ===================

In [ ]:
import json, numpy as np
idx = json.load(open('./limo_atf/great_data_doc/index.json','r',encoding='utf-8'))
k,info = next(iter(idx['docs'].items()))
A = np.load(info['path'])
print(k, A.shape, A.dtype)   # 期望: (行数, 128) float16/float32


### ***No PCA, Trying raw values (768 dim)***

In [1]:
# ===== segment -> line-level raw 768D sidecar =====
import os, json, hashlib
import numpy as np
from collections import defaultdict
from tqdm.notebook import tqdm
from datetime import datetime

RAW_EMB_PATH = "./limo_atf/segment_python_atfeature_raw_embeddings.jsonl"  # 注意跟上面保持一致
OUT_DIR = "./limo_atf/great_data_raw"   # 新目录，别覆盖原来的
INDEX_JSON = os.path.join(OUT_DIR, "index.json")

EMB_DIM = 768
DTYPE_SAVE = "float32"
OVERLAP_MODE = "average"

In [2]:
os.makedirs(OUT_DIR, exist_ok=True)

def _safe_key(doc_key: str) -> str:
    return hashlib.sha1(doc_key.encode("utf-8")).hexdigest()[:16]

def _as_dtype(arr: np.ndarray, dtype_name: str) -> np.ndarray:
    return arr.astype(np.float16 if dtype_name == "float16" else np.float32, copy=False)

def iter_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            yield json.loads(line)

In [3]:
# 1) 按 doc_key 收集所有 segment
groups = defaultdict(list)
meta = {}

for rec in tqdm(iter_jsonl(RAW_EMB_PATH), desc="Group segments by doc_key (raw)"):
    dk  = rec["doc_key"]
    s, e = int(rec["seg_start"]), int(rec["seg_end"])
    vec = np.asarray(rec["vec_raw"], dtype=np.float32)
    if vec.shape[0] != EMB_DIM:
        raise ValueError(f"Expect {EMB_DIM}-D, got {vec.shape} for doc_key={dk}")
    groups[dk].append((s, e, vec))
    info = meta.setdefault(dk, {"problem_id": rec.get("problem_id", ""), "max_end": -1})
    if e > info["max_end"]:
        info["max_end"] = e


Group segments by doc_key (raw): 0it [00:00, ?it/s]

In [4]:
# 2) segment -> line：和 128 维版本一样，用 average 模式广播
index = {}
for dk, segs in tqdm(groups.items(), desc="Write per-doc raw line npy"):
    L = int(meta[dk]["max_end"]) + 1
    if L <= 0:
        continue

    A = np.zeros((L, EMB_DIM), dtype=np.float32)
    C = np.zeros((L, 1),       dtype=np.int32)

    for s, e, vec in segs:
        s = max(0, s)
        e = min(L - 1, e)
        if e < s:
            continue
        A[s:e+1, :] += vec
        C[s:e+1, :] += 1

    mask = (C > 0)
    A[mask[:, 0]] /= C[mask][:, None]

    key = _safe_key(dk)
    npy_path = os.path.join(OUT_DIR, f"{key}.npy")
    np.save(npy_path, _as_dtype(A, DTYPE_SAVE))

    index[dk] = {
        "problem_id": meta[dk]["problem_id"],
        "n_lines": int(L),
        "dim": EMB_DIM,
        "dtype": DTYPE_SAVE,
        "path": npy_path
    }

Write per-doc raw line npy:   0%|          | 0/2752 [00:00<?, ?it/s]

In [5]:
with open(INDEX_JSON, "w", encoding="utf-8") as wf:
    json.dump({
        "created_at": datetime.utcnow().isoformat()+"Z",
        "overlap_mode": OVERLAP_MODE,
        "docs": index
    }, wf, ensure_ascii=False, indent=2)

print(f"[DONE] raw-docs: {len(index)}")
print(f"[INDEX] {INDEX_JSON}")

[DONE] raw-docs: 2752
[INDEX] ./limo_atf/great_data_raw/index.json
